In [1]:
import tensorflow as tf
import glob

from config import TRANSFORMED_TRAIN_ANNOTATIONS_PATH,IMAGE_SIZE
from models.six_stage_linear_model import ModelMaker
import dataset_functions



In [2]:
import visualizations as v
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import importlib as il
il.reload(dataset_functions)

<module 'dataset_functions' from 'C:\\Users\\flash\\Project\\Yet-Another-Openpose-Implmentation\\dataset_functions.py'>

---
# Make dataset

In [4]:
tfrecord_files=glob.glob(TRANSFORMED_TRAIN_ANNOTATIONS_PATH+"-*.tfrecords")
ds_raw = tf.data.TFRecordDataset(tfrecord_files)

In [5]:
TF_parser=dataset_functions.TFrecordParser()
ds_parsed=ds_raw.map(TF_parser.read_tfrecord)

In [6]:
label_transformer=dataset_functions.LabelTransformer()

@tf.function
def make_label_tensors(elem):
    idd=elem['id']
    kpt_tr=label_transformer.keypoints_spots_vmapfn(elem['kpts'])
    paf_tr=label_transformer.joints_PAFs(elem['joints'])
    
    image_raw=elem["image_raw"]
    image=tf.image.decode_jpeg(image_raw,channels=3)
    image=tf.image.convert_image_dtype(image,dtype=tf.float32)
    image=tf.image.resize(image,IMAGE_SIZE)
    return image,(paf_tr,kpt_tr),idd

In [7]:
@tf.function
def place_training_labels(image,labels,idd):
    paf_tr=labels[0]
    kpt_tr=labels[1]
    return image,(paf_tr,paf_tr,paf_tr,paf_tr,kpt_tr,kpt_tr) #this should match the model outputs, and is different for each model

In [8]:
#ds=parsed_ds.cache()
ds_tnlables=ds_parsed.map(make_label_tensors)
ds_tnlabels_arranged=ds_tnlables.map(place_training_labels)

In [9]:
ds_batched=ds_tnlabels_arranged.batch(16)
#shuffle,repeat,prefetch, etc here
ds=ds_batched.repeat()

---
# Model

In [10]:
#@tf.function
def mse_2d_loss(y_true, y_pred):
    pixel_losses=tf.keras.losses.mean_squared_error(y_true, y_pred)
    return tf.math.reduce_mean(pixel_losses,axis=-1)

In [11]:
model_maker=ModelMaker()
train_model,test_model=model_maker.create_models()

In [12]:
train_model.compile(optimizer=tf.keras.optimizers.SGD(),
                    loss=mse_2d_loss,
                    #metrics=["acc"]
                   )

---
Actually training

In [13]:
train_model.fit(ds,epochs=1,steps_per_epoch=2)

Train for 2 steps
y_true-Tensor("loss/stage1paf_output_loss/Shape:0", shape=(4,), dtype=int32) - y_pred-Tensor("loss/stage1paf_output_loss/Shape_1:0", shape=(4,), dtype=int32)
y_true-Tensor("loss/stage2paf_output_loss/Shape:0", shape=(4,), dtype=int32) - y_pred-Tensor("loss/stage2paf_output_loss/Shape_1:0", shape=(4,), dtype=int32)
y_true-Tensor("loss/stage3paf_output_loss/Shape:0", shape=(4,), dtype=int32) - y_pred-Tensor("loss/stage3paf_output_loss/Shape_1:0", shape=(4,), dtype=int32)
y_true-Tensor("loss/stage4paf_output_loss/Shape:0", shape=(4,), dtype=int32) - y_pred-Tensor("loss/stage4paf_output_loss/Shape_1:0", shape=(4,), dtype=int32)
y_true-Tensor("loss/stage5heatmap_output_loss/Shape:0", shape=(4,), dtype=int32) - y_pred-Tensor("loss/stage5heatmap_output_loss/Shape_1:0", shape=(4,), dtype=int32)
y_true-Tensor("loss/stage6heatmap_output_loss/Shape:0", shape=(4,), dtype=int32) - y_pred-Tensor("loss/stage6heatmap_output_loss/Shape_1:0", shape=(4,), dtype=int32)
1/2 [=============

In [17]:
ds

<RepeatDataset shapes: ((None, 368, 368, 3), ((None, 38, 46, 46, 2), (None, 38, 46, 46, 2), (None, 38, 46, 46, 2), (None, 38, 46, 46, 2), (None, 17, 46, 46), (None, 17, 46, 46))), types: (tf.float32, (tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32))>

In [66]:
old=tf.range(38*46*46)
old=tf.reshape(old,(19,46,46,2))
test=old[3,4,4,1]
test

<tf.Tensor: id=11878, shape=(), dtype=int32, numpy=13073>

In [76]:
tr=tf.transpose(old,[1,2,0,3])
#(46,46,19,2)
x=tr[...,0]
y=tr[...,1]
# even=np.arange(0,38,2)
# odd=np.arange(1,38,2)
# indices=[even,odd]
# r=tf.dynamic_stitch(indices,[x,y])
# t=tf.transpose(r,(1,2,0))

# test2=t[4,4,7]
# test2
r=tf.concat((x,y),axis=-1)
#r
r[4,4,19+3]
#tf.transpose(r,(1,2,0))

<tf.Tensor: id=11966, shape=(), dtype=int32, numpy=13073>

In [72]:
x

<tf.Tensor: id=11909, shape=(19, 46, 46), dtype=int32, numpy=
array([[[    0,     2,     4, ...,    86,    88,    90],
        [   92,    94,    96, ...,   178,   180,   182],
        [  184,   186,   188, ...,   270,   272,   274],
        ...,
        [ 3956,  3958,  3960, ...,  4042,  4044,  4046],
        [ 4048,  4050,  4052, ...,  4134,  4136,  4138],
        [ 4140,  4142,  4144, ...,  4226,  4228,  4230]],

       [[ 4232,  4234,  4236, ...,  4318,  4320,  4322],
        [ 4324,  4326,  4328, ...,  4410,  4412,  4414],
        [ 4416,  4418,  4420, ...,  4502,  4504,  4506],
        ...,
        [ 8188,  8190,  8192, ...,  8274,  8276,  8278],
        [ 8280,  8282,  8284, ...,  8366,  8368,  8370],
        [ 8372,  8374,  8376, ...,  8458,  8460,  8462]],

       [[ 8464,  8466,  8468, ...,  8550,  8552,  8554],
        [ 8556,  8558,  8560, ...,  8642,  8644,  8646],
        [ 8648,  8650,  8652, ...,  8734,  8736,  8738],
        ...,
        [12420, 12422, 12424, ..., 12506

In [56]:
x=old[...,0]
y=old[...,1]
tf.stack((x,y),axis=-1)

<tf.Tensor: id=11824, shape=(19, 46, 46, 2), dtype=int32, numpy=
array([[[[    0,     1],
         [    2,     3],
         [    4,     5],
         ...,
         [   86,    87],
         [   88,    89],
         [   90,    91]],

        [[   92,    93],
         [   94,    95],
         [   96,    97],
         ...,
         [  178,   179],
         [  180,   181],
         [  182,   183]],

        [[  184,   185],
         [  186,   187],
         [  188,   189],
         ...,
         [  270,   271],
         [  272,   273],
         [  274,   275]],

        ...,

        [[ 3956,  3957],
         [ 3958,  3959],
         [ 3960,  3961],
         ...,
         [ 4042,  4043],
         [ 4044,  4045],
         [ 4046,  4047]],

        [[ 4048,  4049],
         [ 4050,  4051],
         [ 4052,  4053],
         ...,
         [ 4134,  4135],
         [ 4136,  4137],
         [ 4138,  4139]],

        [[ 4140,  4141],
         [ 4142,  4143],
         [ 4144,  4145],
         ...,
  

In [46]:
even,odd

(array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32,
        34, 36]),
 array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33,
        35, 37]))

In [73]:
it=iter(ds)

In [74]:
sample_tensor=next(it)
sample_tensor

(<tf.Tensor: id=28529, shape=(16, 368, 368, 3), dtype=float32, numpy=
 array([[[[0.3875934 , 0.66330737, 0.65486217],
          [0.3928799 , 0.6752328 , 0.6634681 ],
          [0.38378927, 0.6661422 , 0.6543775 ],
          ...,
          [0.19952066, 0.27898782, 0.2801771 ],
          [0.1932628 , 0.26672512, 0.27036053],
          [0.19670658, 0.27467483, 0.27757272]],
 
         [[0.40064937, 0.67636335, 0.6757613 ],
          [0.3918364 , 0.6741893 , 0.66704786],
          [0.39799628, 0.6803492 , 0.6687826 ],
          ...,
          [0.21934432, 0.3466248 , 0.3086026 ],
          [0.22671112, 0.35353172, 0.31550956],
          [0.23025654, 0.3414582 , 0.3123842 ]],
 
         [[0.39287412, 0.67421675, 0.6708003 ],
          [0.37487087, 0.6602866 , 0.65636504],
          [0.37688148, 0.66077936, 0.65594155],
          ...,
          [0.23374228, 0.3535206 , 0.33416852],
          [0.24031316, 0.35586068, 0.3383021 ],
          [0.23748069, 0.35858053, 0.3356978 ]],
 
         ...

In [26]:
sample_tensor=next(iter(ds_tnlabels_arranged))
img=sample_tensor[0]
pafs=sample_tensor[1][0]
kpts=sample_tensor[1][5]

InvalidArgumentError: Cannot add tensor to the batch: number of elements does not match. Shapes are: [tensor]: [368,368,1], [batch]: [368,368,3] [Op:IteratorGetNextSync]

In [19]:
#v.show_pafs_kpts_img(img.numpy(),paf.numpy(),kpt.numpy(),1,1) #can be used to draw the tensor data